# Corrected LSTM

My previous version was doing the recursion incorrectly, so this is me checking with a corrected version of recursion and using all the features and then I'll go build the class a la Marcos and do the grid search to tune parameters

In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.metrics import mean_squared_error as mse

import sys
sys.path.insert(0, '../')
from gw_tools.gw_plot import gw_plot
from gw_tools import data_prep

2023-11-20 11:44:28.385162: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 11:44:29.674288: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 11:44:29.674374: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 11:44:29.678638: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 11:44:30.450796: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 11:44:30.452507: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
WINDOW_SIZE = 6 # this will be a tuneable parameter later

def df_to_X_multi(df, window_size=WINDOW_SIZE):
    df_as_np = df.to_numpy()
    X = []
    for i in range(len(df_as_np)-window_size):
        row = [a for a in df_as_np[i:i+window_size]]
        X.append(row)
    return np.array(X)

In [7]:
df = data_prep.select_features(pd.read_pickle('../data/pickled_data/AFL259_short.pkl'))
df = data_prep.add_toy_signal(df).set_index('date')

df.head(3)

,avg_well_depth,gage_ht,discharge_cfs,prcp,temp_avg,hum_avg,hPa_avg,wind_avg,gust_avg,prcp_lag_45D,year_sin,year_cos
date,,,,,,,,,,,,
2005-08-21,99.059167,17.02,600.0,0.0,75.903750,44.166667,1015.333333,7.492083,0.000000,0.00,-0.756711,-0.653749
2005-08-22,99.163333,17.02,599.0,0.0,72.854167,43.041667,1012.458333,10.093333,0.717500,0.16,-0.767845,-0.640635
2005-08-23,99.155000,17.02,598.0,0.0,65.272083,46.041667,1010.000000,14.121250,6.141667,0.72,-0.778752,-0.627332
